# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Msc. BDS - M7 Second Semester Project** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

## <span style='color:#2656a3'> 🗒️ The notebook is divided into the following sections:
1. Parsing new data.
2. Inserting the new data into the Feature Store.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

We start by accessing the folder we have created that holds the functions (incl. live API calls and data preprocessing) we need for electricity prices and weather measures. Then, we proceed to import some of the necessary libraries needed for this notebook and warnings to avoid unnecessary distractions and keep output clean.

In [1]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
# This is the functions we have created to generate features for electricity prices and weather measures
from features import electricity_prices, weather_measures

# We go back into the notebooks folder
%cd pipeline

/Users/tobiasmjensen/Documents/aau_bds/m7_second-semester-project/bds_m7_second-semester-project
/Users/tobiasmjensen/Documents/aau_bds/m7_second-semester-project/bds_m7_second-semester-project/pipeline


In [2]:
# Importing pandas for data handling
import pandas as pd
from datetime import datetime, timedelta

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

## <span style='color:#2656a3'> 🪄 Parsing New Data
To fetch non-historical electricity prices we are setting `historical` to `False`. 

In order to provide real time weather measures, a weather forecast measure for the next 5 days is being fetched.

There are of course no changes to the calendar data, and therefore no new data is retrieved from it.

### <span style="color:#2656a3;">💸 Electricity Prices per day from Energinet

In [3]:
# Fetching non-historical electricity prices for area DK1
electricity_price_df = electricity_prices.electricity_prices(
    historical=False,
    area=["DK1"]
)

In [4]:
# Display the electricity dataframe
electricity_price_df

,timestamp,datetime,date,hour,dk1_spotpricedkk_kwh
0,1716508800000,2024-05-24 00:00:00,2024-05-24,0,0.76987
1,1716512400000,2024-05-24 01:00:00,2024-05-24,1,0.70234
2,1716516000000,2024-05-24 02:00:00,2024-05-24,2,0.67003
3,1716519600000,2024-05-24 03:00:00,2024-05-24,3,0.65831
4,1716523200000,2024-05-24 04:00:00,2024-05-24,4,0.68503
5,1716526800000,2024-05-24 05:00:00,2024-05-24,5,0.77032
6,1716530400000,2024-05-24 06:00:00,2024-05-24,6,0.92412
7,1716534000000,2024-05-24 07:00:00,2024-05-24,7,0.91091
8,1716537600000,2024-05-24 08:00:00,2024-05-24,8,0.86487
9,1716541200000,2024-05-24 09:00:00,2024-05-24,9,0.79174


In [5]:
# Display the information of the electricity dataframe
electricity_price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             24 non-null     int64         
 1   datetime              24 non-null     datetime64[ns]
 2   date                  24 non-null     datetime64[ns]
 3   hour                  24 non-null     int64         
 4   dk1_spotpricedkk_kwh  24 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 1.1 KB


### <span style="color:#2656a3;">🪟 Rolling window for Electricity Prices

In [6]:
# Fetching non-historical electricity prices and rolling window for area DK1
electricity_price_window_df = electricity_prices.electricity_prices_window(
    historical=False,
    area=["DK1"],
    end=datetime.now().date() + timedelta(days=(7*1)) 
)

In [7]:
# Display the electricity_window dataframe
electricity_price_window_df

,timestamp,datetime,prev_1w_mean,prev_2w_mean,prev_4w_mean,prev_6w_mean,prev_8w_mean,prev_12w_mean
38520,1716508800000,2024-05-24 00:00:00,0.400126,0.323872,0.392901,0.428766,0.411718,0.429602
38521,1716512400000,2024-05-24 01:00:00,0.401696,0.322945,0.392663,0.428869,0.411920,0.429625
38522,1716516000000,2024-05-24 02:00:00,0.403297,0.322188,0.392423,0.428965,0.412158,0.429646
38523,1716519600000,2024-05-24 03:00:00,0.404921,0.321485,0.392183,0.429089,0.412408,0.429667
38524,1716523200000,2024-05-24 04:00:00,0.406546,0.320771,0.391909,0.429220,0.412646,0.429688
...,...,...,...,...,...,...,...,...
38707,1717182000000,2024-05-31 19:00:00,NaN,0.431920,0.401711,0.432395,0.412202,0.421164
38708,1717185600000,2024-05-31 20:00:00,NaN,0.433180,0.400843,0.432333,0.412211,0.421104
38709,1717189200000,2024-05-31 21:00:00,NaN,0.434479,0.400046,0.432258,0.412223,0.421070
38710,1717192800000,2024-05-31 22:00:00,NaN,0.435878,0.399461,0.432203,0.412239,0.421036


In [8]:
# Display the information of the electricity_window dataframe
electricity_price_window_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 38520 to 38711
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timestamp      192 non-null    int64         
 1   datetime       192 non-null    datetime64[ns]
 2   prev_1w_mean   167 non-null    float64       
 3   prev_2w_mean   192 non-null    float64       
 4   prev_4w_mean   192 non-null    float64       
 5   prev_6w_mean   192 non-null    float64       
 6   prev_8w_mean   192 non-null    float64       
 7   prev_12w_mean  192 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 13.5 KB


### <span style="color:#2656a3;"> 🌈 Forecast Weather Measures from Open Meteo

In [9]:
# Fetching weather forecast measures for the next 5 days
forecast_weather_df = weather_measures.forecast_weather_measures(
    forecast_length=5
)

In [10]:
# Display the weather forecast dataframe
forecast_weather_df

,timestamp,datetime,date,hour,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1716508800000,2024-05-24 00:00:00,2024-05-24,0,14.4,94.0,0.0,0.0,0.0,3.0,100.0,11.5,21.2
1,1716512400000,2024-05-24 01:00:00,2024-05-24,1,14.4,92.0,0.0,0.0,0.0,3.0,100.0,12.6,22.3
2,1716516000000,2024-05-24 02:00:00,2024-05-24,2,14.1,89.0,0.0,0.0,0.0,3.0,100.0,13.3,23.8
3,1716519600000,2024-05-24 03:00:00,2024-05-24,3,13.9,90.0,0.0,0.0,0.0,3.0,87.0,9.0,20.9
4,1716523200000,2024-05-24 04:00:00,2024-05-24,4,13.9,93.0,0.0,0.0,0.0,2.0,74.0,11.9,20.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1716922800000,2024-05-28 19:00:00,2024-05-28,19,12.8,87.0,0.0,0.0,0.0,0.0,47.0,3.9,11.2
116,1716926400000,2024-05-28 20:00:00,2024-05-28,20,12.2,90.0,0.0,0.0,0.0,0.0,23.0,3.4,9.4
117,1716930000000,2024-05-28 21:00:00,2024-05-28,21,11.7,92.0,0.0,0.0,0.0,0.0,0.0,3.8,7.2
118,1716933600000,2024-05-28 22:00:00,2024-05-28,22,11.2,93.0,0.0,0.0,0.0,0.0,1.0,4.2,6.8


In [11]:
# Display the information of the weather forecast dataframe
forecast_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             120 non-null    int64         
 1   datetime              120 non-null    datetime64[ns]
 2   date                  120 non-null    datetime64[ns]
 3   hour                  120 non-null    int64         
 4   temperature_2m        120 non-null    float64       
 5   relative_humidity_2m  120 non-null    float64       
 6   precipitation         120 non-null    float64       
 7   rain                  120 non-null    float64       
 8   snowfall              120 non-null    float64       
 9   weather_code          120 non-null    float64       
 10  cloud_cover           120 non-null    float64       
 11  wind_speed_10m        120 non-null    float64       
 12  wind_gusts_10m        120 non-null    float64       
dtypes: datetime64[ns](2)

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

We connect to Hopsworks Feature Store so we can access the Feature Groups and upload the new data into the Feature Groups.

In [12]:
# Importing the hopsworks module for interacting with the Hopsworks platform
import hopsworks

# Logging into the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/554133
Connected. Call `.close()` to terminate connection gracefully.


In [13]:
# Retrieve the feature groups
electricity_price_fg = fs.get_feature_group(
    name="electricity_spot_prices",
    version=1,
)

# Retrieve the feature groups
electricity_price_window_fg = fs.get_feature_group(
    name="electricity_spot_price_window",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather_measurements",
    version=1,
)

### <span style="color:#2656a3;"> ⬆️ Uploading new data to the Feature Store
Here we upload the new data to the retrieved Feature groups by using the `insert` function.

In [14]:
# Inserting the electricity_price_df into the feature group named electricity_fg
electricity_price_fg.insert(electricity_price_df, 
                      write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/24 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: electricity_spot_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/electricity_spot_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x308aa0ad0>, None)

In [15]:
# Inserting the electricity_price_window_df into the feature group named electricity_price_window_fg
electricity_price_window_fg.insert(electricity_price_window_df, 
                      write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/192 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: electricity_spot_price_window_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/electricity_spot_price_window_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x308ad77d0>, None)

In [16]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(forecast_weather_df, 
                  write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/120 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/554133/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x308ad3950>, None)

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 03: Training Pipeline </span>

Next we will create a feature view and training dataset. Further we will train a model and save it in model registry.